In [60]:
# import dependencies

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [63]:
computing_deals = [] # create empty list to hold data

# loop through all the webpages listing computing deals products
for i in range(1, 51):
     
     # set url of website to be scraped
     SCRAPE_URL = f'https://www.jumia.co.ke/mlp-black-friday-h-computing-deals/?page={i}#catalog-listing'

     # send GET request to specified url; parse the data retrieved
     response = requests.get(SCRAPE_URL)
     soup = BeautifulSoup(response.content, 'html.parser')
     soup = BeautifulSoup(soup.prettify(), 'html.parser')


     # DATA EXTRACTION BEGINS HERE:
     # access all <article> tags with product data
     articles = soup.find_all('article', class_ = 'prd')

     # go through all <article> tags holding the data of interest
     for article in articles:
          
          # get product title/description
          prd_title = article.find('h3').text.strip()
          
          # get product current price; convert type to float
          curr_price = article.find('div', class_ = 'prc').text.strip().split()[1]
          curr_price = float(curr_price.replace(',', ''))
          
          
          # get product old price; convert type to float
          if article.find('div', class_ = 'old'):
               old_price = article.find('div', class_ = 'old').text.strip().split()[1]
               old_price = float(old_price.replace(',', ''))
          else: # handle cases without recorded old price values
               old_price = None
          
          
          # get product discount rate
          if article.find('div', class_ = '_dsct'):
               discount = article.find('div', class_ = '_dsct').text.strip()
               discount = abs(int(discount.replace('%', '')))
          else: # handle cases without recorded discount values
               discount = None
          
          
          # get product star rating; convert type to float
          if article.find('div', class_ = 'stars'):
               star_rating = float(article.find('div', class_ = 'stars').text.strip().split()[0])
          else: # handle cases without recorded star rating values
               star_rating = None
          
          
          # get count of verified ratings
          if article.find('div', class_ = 'rev'):
               verified_rating_count = int(article.find('div', class_ = 'rev').text.strip().split()[4][1:-1])
          else: # handle cases without recorded verified rating count values
               verified_rating_count = None
          
          
          # populate the empty list with extracted data
          computing_deals.append([prd_title, curr_price, old_price, discount, star_rating, verified_rating_count])

In [ ]:
# check if list is populated correctly
computing_deals

In [ ]:
# create new pandas DataFrame
# populate with data from computing_deals list
df = pd.DataFrame(computing_deals, 
                  columns=['Product', 'Current Price', 'Old Price', 'Discount in %', 'Star Rating', 'Verified Rating Count'])

df

In [59]:
# save generated dataframe to csv file format
df.to_csv('computing_deals.csv')